In [1]:
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display, clear_output

Created a pandas dataframe from Postgres database  
df = pd.read_sql_query('select * from "tweets"', con=engine)  
Saved as csv

In [2]:
df = pd.read_csv('df.csv')

In [3]:
df.head()

,id,date_created,tweet_id,tweet,hashtags
0,1,Fri Jul 31 17:17:54 +0000 2020,1289248935672868866,And you lying to the few viewers you have abou...,{CuomoPrivilege}
1,2,Fri Jul 31 17:17:54 +0000 2020,1289248939267379200,"As we enter the fifth month of masks, social d...",{}
2,3,Fri Jul 31 17:18:04 +0000 2020,1289248977813004289,Skin So Soft Original Dry Oil Spray - Limited ...,"{avon,skinsosoft,skincare,dryoilspray,jojobaoi..."
3,4,Fri Jul 31 17:18:05 +0000 2020,1289248985371140096,"If #Covid_19 was visible as a green cloud, peo...","{Covid_19,CovidHearing,fridaymorning,lockdowne..."
4,5,Fri Jul 31 17:18:05 +0000 2020,1289248985547300864,"From the 1st day of the pandemic, some of us i...",{}


Function the chooses a number of text items to annotate.
* consider random selection
* consider if already selected, how do you not select it again (if id not in annotation_df.id)
* use widgets to impute data into the function text widget and slider

In [4]:
def subset_df(df, primary_key, text_column, num_records):
    '''
    Subsets the dataframe by the primary key and text columns with a chosen number of records.
    '''
    subset_df = df[[primary_key, text_column]][:num_records]
    return subset_df

In [5]:
subset_df = subset_df(df, 'id', 'tweet', 3) 
subset_df.head()

,id,tweet
0,1,And you lying to the few viewers you have abou...
1,2,"As we enter the fifth month of masks, social d..."
2,3,Skin So Soft Original Dry Oil Spray - Limited ...


In [6]:
try:
    annotations = annotations
    
except:
    columns = ['id','label','date_annotated']
    annotations = pd.DataFrame(columns=columns)
    
annotation_index = 0

label = []
date_annotated = []

def load_annotations():
    '''
    Displays texts to be annotated and loads user inputted labels and timestamp to dataframe.
    '''

    instructions = widgets.HTML("<b>Click on the appropriate sentiment for the text below:</b>")
    text = HTML(subset_df.iloc[annotation_index, 1])
    positive_button = widgets.Button(description='Positive')
    negative_button = widgets.Button(description='Negative')
    neutral_button = widgets.Button(description='Neutral')
    skip_button = widgets.Button(description='Skip')
    confirm_button = widgets.Button(description='Confirm selection')
    redo_button = widgets.Button(description='Try again')
    sentiment_buttons = widgets.HBox([positive_button, negative_button, neutral_button, skip_button])
    confirmation_buttons = widgets.HBox([confirm_button, redo_button])
    output = widgets.Output()

    display(instructions, text, sentiment_buttons, confirmation_buttons, output)
    confirmation_buttons.layout.visibility = 'hidden'    


    def sentiment_button_clicked(button):
        label.append(str(button.description).lower())
        date_annotated.append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        
        with output:
            clear_output(True)
            sentiment_buttons.layout.visibility = 'hidden'
            print('The text has', str(button.description).lower(), 'sentiment.')
            confirmation_buttons.layout.visibility = 'visible'

    positive_button.on_click(sentiment_button_clicked)
    negative_button.on_click(sentiment_button_clicked)
    neutral_button.on_click(sentiment_button_clicked)
    
    def skip_button_clicked(button):
        label.append(None)
        date_annotated.append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        
        with output:
            clear_output(True)
            sentiment_buttons.layout.visibility = 'hidden'
            print('This action will add a Null value to the label for this record.')
            confirmation_buttons.layout.visibility = 'visible'
        
    skip_button.on_click(skip_button_clicked)

    def confirm_button_clicked(button):
        global annotation_index
        global annotations
        if annotation_index < len(subset_df) - 1:
            annotation_index += 1
            clear_output(True)
            load_annotations()
        else:
            clear_output(True)
            display(HTML('<h3>Annotations are done.</h3><br><i>annotation_df = new_df will bring up your annotations.</i>'))
            new_df = pd.DataFrame({
                'id': subset_df.iloc[:, 0].tolist(),
                'label': label,
                'date_annotated': date_annotated
            })
            annotations = annotations.append(new_df)
            

    confirm_button.on_click(confirm_button_clicked)
    
    def redo_button_clicked(button):
        del label[-1]
        del date_annotated[-1]
        with output:
            clear_output(True)
            sentiment_buttons.layout.visibility = 'visible'
            print('Please try again.')
            confirmation_buttons.layout.visibility = 'hidden'
            

    redo_button.on_click(redo_button_clicked)

This will provide a user interface to annotate tweets.  
* add a progress bar to show progress of annotations

In [7]:
load_annotations()

HTML(value='<h3>Annotations are done.</h3><br><i>annotation_df = new_df will bring up your annotations.</i>')

In [8]:
annotations

,id,label,date_annotated
0,1,positive,2020-08-07 17:45:45
1,2,neutral,2020-08-07 17:45:49
2,3,None,2020-08-07 17:45:53
